# Supplementary Material for Assessing Multi-site rs-fMRI Harmonization using Information Theory

Part of this code was taken and adapted from the following repositories:

*   [https://github.com/dagush/WholeBrain](https://github.com/dagush/WholeBrain)
*   [https://gitlab.com/cristophersfr/fisher-networks](https://gitlab.com/cristophersfr/fisher-networks)

# Configurations

In [ ]:
# Download libraries
import sys
!{sys.executable} -m pip install nilearn dask[dataframe] neuroCombat neuroHarmonize

# Harmonization
from neuroCombat import neuroCombat
import covbat
import patsy
# Kruskal-Wallis
from scipy.stats import kruskal
# Code optimization
from numba import jit
# Paths
from pathlib import Path
# Folder change
import os
# Time measurement
from timeit import default_timer as timer
# Data management
import numpy as np
import pandas as pd
import dask.dataframe
import scipy.io
# Plots
import matplotlib
import matplotlib.pyplot as plt
import altair as alt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.6 MB 20.6 MB/s 
     |████████████████████████████████| 38.1 MB 1.2 MB/s 
     |████████████████████████████████| 140 kB 61.3 MB/s 
  Created wheel for neuroCombat: filename=neuroCombat-0.2.12-py3-none-any.whl size=6371 sha256=cde83b37f5dbebb3be8a39d3e52e6d7ee57fa618710c63dd45c61ff3f8b89e4a
  Stored in directory: /root/.cache/pip/wheels/69/cb/ed/6b85d2d01fe75ac43fff6cb93d2da82efcff02527595198ae2
Successfully built neuroCombat
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [ ]:
# Define working folder
folder = Path('')             # ADD NAME OF THE FOLDER
os.chdir(folder)

# Read and process dataset

## Read datasets

IMPORTANT: run only the code of the desired dataset, do not work with more than one dataset at a time.

### IMPAC

In [ ]:
# UNCOMMENT AND RUN ONLY ONCE

# Download all atlases
# from download_data import fetch_fmri_time_series
# fetch_fmri_time_series()

In [ ]:
def _read_data_impac(path, filename):
    subject_id = pd.read_csv(os.path.join(path, 'data', filename), header=None)
    # read the list of the subjects
    df_participants = pd.read_csv(os.path.join(path, 'data',
                                               'participants.csv'),
                                  index_col=0)
    df_participants.columns = ['participants_' + col
                               for col in df_participants.columns]
    # load the structural and functional MRI data
    df_anatomy = pd.read_csv(os.path.join(path, 'data', 'anatomy.csv'),
                             index_col=0)
    df_anatomy.columns = ['anatomy_' + col
                          for col in df_anatomy.columns]
    df_fmri = pd.read_csv(os.path.join(path, 'data', 'fmri_filename.csv'),
                          index_col=0)
    df_fmri.columns = ['fmri_' + col
                       for col in df_fmri.columns]
    # load the QC for structural and functional MRI data
    df_anatomy_qc = pd.read_csv(os.path.join(path, 'data', 'anatomy_qc.csv'),
                                index_col=0)
    df_fmri_qc = pd.read_csv(os.path.join(path, 'data', 'fmri_qc.csv'),
                             index_col=0)
    df_fmri_tr = pd.read_csv(os.path.join(path, 'data',
                                          'fmri_repetition_time.csv'),
                             index_col=0)
    # rename the columns for the QC to have distinct names
    df_anatomy_qc = df_anatomy_qc.rename(columns={"select": "anatomy_select"})
    df_fmri_qc = df_fmri_qc.rename(columns={"select": "fmri_select"})

    X = pd.concat([df_participants, df_anatomy, df_anatomy_qc, df_fmri,
                   df_fmri_qc, df_fmri_tr], axis=1)
    X = X.loc[subject_id[0]]
    y = X['participants_asd']
    X = X.drop('participants_asd', axis=1)

    return X, y.values

In [ ]:
# Obtain data
data_train, labels_train = _read_data_impac(folder/'autism-master/autism-master','train.csv')
data_test, labels_test = _read_data_impac(folder/'autism-master/autism-master','test.csv')

ds_flag = 'impac'

### ABIDE

In [ ]:
# UNCOMMENT AND RUN ONLY ONCE PER ATLAS

# Download one atlas
# from download_data import fetch_fmri_time_series
# !python download_abide_preproc.py -d rois_aal -p dparsf -s nofilt_noglobal -o 'abide'

In [ ]:
def _read_data_abide(path):
    # read the list of the subjects
    df_data = pd.read_csv(path)
    
    y = df_data['DX_GROUP']
    X = df_data.drop('DX_GROUP', axis=1)

    return X, y

In [ ]:
# Obtain data
data_train, labels_train = _read_data_abide(folder/'abide/data.csv')

ds_flag = 'abide'

### ADHD200

In [ ]:
# THE DATA MUST BE DOWNLOADED MANUALLY FROM https://www.nitrc.org/frs/?group_id=383 (ADHD200 Preproc Athena time courses)

In [ ]:
def _read_data_adhd200(path):
    # read the list of the subjects

    df_data = pd.read_csv(path, dtype='string')
    df_data['Site'] = df_data['Site'].astype(float)
    df_data['Gender'] = df_data['Gender'].astype(float)
    df_data['Age'] = df_data['Age'].astype(float)
    
    y = df_data['DX']
    X = df_data.drop('DX', axis=1)

    return X, y

In [ ]:
# Obtain data
data_train, labels_train = _read_data_adhd200(folder/'ADHD200/data.csv')

ds_flag = 'adhd200'

### SRPBS

In [ ]:
# IN ORDER TO OBTAIN THIS DATASET, YOU MUST FIRST ASK THE AUTHORS FOR PERMISSION AT https://bicr.atr.jp/dcn/en/download/harmonization/. 
# THEN PROCESS IN MATLAB AND SAVE THE RESULTS IN /FOLDER/SRPBS/COR_SRPBS_ALL_ORG.mat

In [ ]:
def _read_data_srpbs(path):
    # read the list of the subjects
    data = np.array(scipy.io.loadmat(path)['DATA_SUB'])
    df_data = pd.DataFrame(data, columns=['DX_GROUP', 'SITE_ID', 'AGE_AT_SCAN', 'SEX'])

    y = df_data['DX_GROUP']
    X = df_data.drop('DX_GROUP', axis=1)

    return X, y

In [ ]:
# Obtain data
data_train, labels_train = _read_data_srpbs(folder/'SRPBS/COR_SRPBS_ALL_ORG.mat')

ds_flag = 'srpbs'

## Conectivity

### Initialization and filtering

In [ ]:
TR = 2.                           # sampling interval
k = 2                             # 2nd order butterworth filter
flp = 0.04                        # lowpass frequency of filter
fhi = 0.07                        # highpass
from scipy.signal import butter, detrend, filtfilt
import numpy.matlib as mtlib

def demean(x,dim=0):
    dims = x.size
    return x - mtlib.tile(np.mean(x,dim), dims)  # repmat(np.mean(x,dim),dimrep)

In [ ]:
def BandPassFilter(boldSignal, removeStrongArtefacts=True):
    # Convenience method to apply a filter (always the same one) to all areas in a BOLD signal. For a single,
    # isolated area evaluation, better use the method below.
    (N, Tmax) = boldSignal.shape
    fnq = 1./(2.*TR)              # Nyquist frequency
    Wn = [flp/fnq, fhi/fnq]                                   # butterworth bandpass non-dimensional frequency
    bfilt, afilt = butter(k,Wn, btype='band', analog=False)   # construct the filter
    # bfilt = bfilt_afilt[0]; afilt = bfilt_afilt[1]  # numba doesn't like unpacking...
    signal_filt = np.zeros(boldSignal.shape)
    for seed in range(N):
        if not np.isnan(boldSignal[seed, :]).any():  # No problems, go ahead!!!
            ts = demean(detrend(boldSignal[seed, :]))  # Probably, we do not need to demean here, detrend already does the job...

            if removeStrongArtefacts:
                ts[ts>3.*np.std(ts)] = 3.*np.std(ts)    # Remove strong artefacts
                ts[ts<-3.*np.std(ts)] = -3.*np.std(ts)  # Remove strong artefacts

            signal_filt[seed,:] = filtfilt(bfilt, afilt, ts, padlen=3*(max(len(bfilt),len(afilt))-1))  # Band pass filter. padlen modified to get the same result as in Matlab
        else:  # We've found problems, mark this region as "problematic", to say the least...
            warnings.warn(f'############ Warning!!! BandPassFilter: NAN found at region {seed} ############')
            signal_filt[seed,0] = np.nan
    return signal_filt

### Phase Interaction Matrix

In [ ]:
def tril_indices_column(N, k=0):
    row_i, col_i = np.nonzero(
        np.tril(np.ones(N), k=k).T)  # Matlab works in column-major order, while Numpy works in row-major.
    Isubdiag = (col_i,
                row_i)  # Thus, I have to do this little trick: Transpose, generate the indices, and then "transpose" again...
    return Isubdiag

@jit(nopython=True)
def adif(a, b):
    if np.abs(a - b) > np.pi:
        c = 2 * np.pi - np.abs(a - b)
    else:
        c = np.abs(a - b)
    return c

In [ ]:
from scipy import signal

discardOffset = 10

@jit(nopython=True)
def FastFor(phases, N, Tmax, dFC, PhIntMatr):
  T = np.arange(discardOffset, Tmax - discardOffset + 1)
  for t in T:
    for i in range(N):
      for j in range(i+1):                    
        dFC[i, j] = np.cos(adif(phases[i, t - 1], phases[j, t - 1]))
        dFC[j, i] = dFC[i, j]
    PhIntMatr[t - discardOffset] = dFC 
  return PhIntMatr

def PhaseInteractionMatrix(ts, applyFilters = True):  # Compute the Phase-Interaction Matrix of an input BOLD signal
    (N, Tmax) = ts.shape
    npattmax = Tmax - (2*discardOffset-1)  # calculates the size of phfcd matrix

    if not np.isnan(ts).any():  # No problems, go ahead!!!
        # Data structures we are going to need...
        phases = np.zeros((N, Tmax))
        dFC = np.zeros((N, N))
        # PhIntMatr = np.zeros((npattmax, int(N * (N - 1) / 2)))  # The int() is not needed, but... (see above)
        PhIntMatr = np.zeros((npattmax, N, N))
        # syncdata = np.zeros(npattmax)

        # Filters seem to be always applied...
        ts_filt = BandPassFilter(ts)  # zero phase filter the data
        # ts_filt = ts
        for n in range(N):
            Xanalytic = signal.hilbert(demean(ts_filt[n, :]))
            phases[n, :] = np.angle(Xanalytic)

        a = FastFor(phases, N, Tmax, dFC, PhIntMatr)
        PhIntMatr = a            

    else:
        warnings.warn('############ Warning!!! PhaseInteractionMatrix.from_fMRI: NAN found ############')
        PhIntMatr = np.array([np.nan])
    return PhIntMatr

In [ ]:
# Calculation of one Phase Interaction Matrix
def obtener_matriz(file, sep=',',skip=0):

  start = timer()

  # Read file
  time_series = np.array(dask.dataframe.read_csv(file, sep=sep))

  # Transpose matrix (time has to be in the x axis)
  ts = time_series.transpose()
  ts = np.array(ts[skip:,:],dtype='float')

  # Phase Interaction Matrix
  correlation_matrix = np.mean(PhaseInteractionMatrix(ts, applyFilters = True), axis=0)  

  end = timer()
  print(end - start)  

  # return tri2mat(correlation_matrix)
  return correlation_matrix

### Processing

#### IMPAC

In [ ]:
# Subject selection
subjects_data = [data_train.iloc[i] for i in range(len(data_train))]

In [ ]:
# Choose one atlas and obtain matrices
sub_path = 'autism-master/autism-master'
correlation_matrices = list()
for i in range(0, len(subjects_data)):
  mat = obtener_matriz(folder/sub_path/subjects_data[i]['fmri_msdl'])     #39
  # mat = obtener_matriz(folder/sub_path/subjects_data[i]['fmri_harvard_oxford_cort_prob_2mm']) #48
  # mat = obtener_matriz(folder/sub_path/subjects_data[i]['fmri_basc064'])  #64
  # mat = obtener_matriz(folder/sub_path/subjects_data[i]['fmri_basc122'])  #122
  # mat = obtener_matriz(folder/sub_path/subjects_data[i]['fmri_basc197'])  #197
  # mat = obtener_matriz(folder/sub_path/subjects_data[i]['fmri_craddock_scorr_mean'])  #249
  # mat = obtener_matriz(folder/sub_path/subjects_data[i]['fmri_power_2011']) #264
  print(i)
  correlation_matrices.append(mat)

correlation_matrices = np.array(correlation_matrices)

#### ABIDE

In [ ]:
# Select path of the desired atlas
sub_path = 'abide/Outputs/dparsf/nofilt_noglobal/rois_tt'

In [ ]:
# Subject selection
subjects_data = list()
drop = list()
for i in range(len(data_train)):
  filename = data_train.iloc[i]['FILE_ID']+'_rois_tt.1D'
  # filename = data_train.iloc[i]['FILE_ID']+'_rois_ho.1D'
  # filename = data_train.iloc[i]['FILE_ID']+'_rois_ez.1D'
  # filename = data_train.iloc[i]['FILE_ID']+'_rois_dosenbach160.1D'
  # filename = data_train.iloc[i]['FILE_ID']+'_rois_cc400.1D'
  # filename = data_train.iloc[i]['FILE_ID']+'_rois_cc200.1D'
  # filename = data_train.iloc[i]['FILE_ID']+'_rois_aal.1D'
  complete_path = folder/sub_path/filename
  if complete_path.is_file():
    subjects_data.append(data_train.iloc[i])
  else:
    drop.append(i)

data_train = data_train.drop(drop)
labels_train = labels_train.drop(drop)

In [ ]:
# Choose one atlas and obtain matrices
correlation_matrices = list()
for i in range(0, len(subjects_data)):
  filename = subjects_data[i]['FILE_ID']+'_rois_tt.1D'
  # filename = subjects_data[i]['FILE_ID']+'_rois_ho.1D'
  # filename = subjects_data[i]['FILE_ID']+'_rois_ez.1D'
  # filename = subjects_data[i]['FILE_ID']+'_rois_dosenbach160.1D'
  # filename = subjects_data[i]['FILE_ID']+'_rois_cc400.1D'
  # filename = subjects_data[i]['FILE_ID']+'_rois_cc200.1D'
  # filename = subjects_data[i]['FILE_ID']+'_rois_aal.1D'
  complete_path = folder/sub_path/filename
  mat = obtener_matriz(complete_path,'\t') 
  print(i)
  correlation_matrices.append(mat)

correlation_matrices = np.array(correlation_matrices)

0.16019193600004655
0
0.1422963040004106
1
0.1553054539999721
2
0.1452814220010623
3
0.17687441899943224
4
0.15350610800123832
5
0.14559439699951326
6
0.14378962800037698
7
0.15224813499844458
8
0.15372398100043938
9
0.16029369799980486
10
0.13425803400059522
11
0.14567347599950153
12
0.13482222300081048
13
0.1381915099991602
14
0.14512187100081064
15
0.15431399999943096
16
0.14327107000099204
17
0.14296336400002474
18
0.14007753199985018
19
0.13898911100113764
20
0.13791236399993068
21
0.13956599099947198
22
0.1357588639984897
23
0.1653600909994566
24
0.15870772100061004
25
0.1457967690002988
26
0.13865371000065352
27
0.1589505440006178
28
0.14063027300107933
29
0.14212488700104586
30
0.14409031200011668
31
0.14235288800045964
32
0.3771888419996685
33
0.5599776189992554
34
0.3542101939983695
35
0.15170576099990285
36
0.15858861900051124
37
0.6167364619996079
38
0.4511395919998904
39
0.19529875499938498
40
0.13747174499985704
41
0.14341995399990992
42
0.578068995000649
43
0.59175552700

#### ADHD200

In [ ]:
# Subject selection
subjects_data = [data_train.iloc[i] for i in range(len(data_train))]

In [ ]:
# Select path of the desired atlas
sub_path = 'ADHD200/AAL'

In [ ]:
# Choose one atlas and obtain matrices
correlation_matrices = list()
for i in range(0, len(subjects_data)):
  subfolder = Path(str(subjects_data[i]['ScanDir ID']))
  # filename = Path('snwmrda'+str(subjects_data[i]['ScanDir ID'])+'_session_1_rest_1_'+'tt_TCs.1D')
  # filename = Path('snwmrda'+str(subjects_data[i]['ScanDir ID'])+'_session_1_rest_1_'+'ho_TCs.1D')
  # filename = Path('snwmrda'+str(subjects_data[i]['ScanDir ID'])+'_session_1_rest_1_'+'cc400_TCs.1D')
  # filename = Path('snwmrda'+str(subjects_data[i]['ScanDir ID'])+'_session_1_rest_1_'+'ez_TCs.1D')
  # filename = Path('snwmrda'+str(subjects_data[i]['ScanDir ID'])+'_session_1_rest_1_'+'cc200_TCs.1D')
  filename = Path('snwmrda'+str(subjects_data[i]['ScanDir ID'])+'_session_1_rest_1_'+'aal_TCs.1D')
  complete_path = folder/sub_path/subfolder/filename
  mat = obtener_matriz(complete_path,'\t',2) 
  print(i)
  correlation_matrices.append(mat)

correlation_matrices = np.array(correlation_matrices)

9.603118491000004
0
0.6587689950000026
1
0.6115135750000036
2
0.7303280530000222
3
1.8984223280000094
4
0.8203111859999979
5
0.8679843180000262
6
0.6543468689999941
7
0.7616835489999971
8
0.6135244830000204
9
0.6342530249999925
10
0.6762754460000053
11
1.032655028999983
12
0.6718174079999812
13
0.8094752979999953
14
0.6734009170000093
15
0.8206400929999802
16
0.8643192740000245
17
0.8360661860000107
18
0.5760749829999838
19
0.5214579290000074
20
0.5708805760000075
21
0.7219804140000008
22
0.7819148759999734
23
0.8578389429999902
24
0.6104308650000121
25
0.4520398290000003
26
0.49062364400001
27
0.5570360659999949
28
0.4976660060000029
29
0.5182758619999959
30
0.5668660750000072
31
0.4505020409999929
32
0.49681381300001703
33
0.5123410930000034
34
0.5978944059999947
35
0.4574195399999894
36
0.5722229170000048
37
0.5350997019999966
38
0.5104160930000035
39
0.4938551899999766
40
0.5188893240000141
41
0.4708944669999937
42
0.55725547199998
43
0.5048659150000105
44
0.4852276659999859
45
0.5

#### SRPBS

In [ ]:
# Convert triangular matrix to symmetric matrix
def tri2mat(tri):

  # Bhaskara of L=n*(n-1)/2
  n = int((1+np.sqrt(1+8*len(tri)))/2)

  mat = np.zeros((n,n))
  mat[np.triu_indices(mat.shape[0], k = 1)] = tri
  mat = mat + mat.T

  # Fill diagonal with ones
  np.fill_diagonal(mat,1)

  return mat

In [ ]:
# Subject selection
subjects_data = [data_train.iloc[i] for i in range(len(data_train))]

In [ ]:
# Read original and harmonized data
correlation_vectors = np.array(scipy.io.loadmat(folder/'SRPBS/COR_SRPBS_ALL_ORG.mat')['X'])
correlation_vectors_har = np.array(scipy.io.loadmat(folder/'SRPBS/COR_SRPBS_ALL_SubtractMeasurementBias.mat')['X'])
# Obtain correlation matrices
correlation_matrices = np.apply_along_axis(tri2mat, 1, correlation_vectors)
correlation_matrices_har = np.apply_along_axis(tri2mat, 1, correlation_vectors_har)

# Harmonization

## No harmonization

In [ ]:
har_flag = 'none'

## ComBat

In [ ]:
# Convert matrices to vectors
s = correlation_matrices.shape
v = correlation_matrices.reshape((s[0],s[1]**2))

# Rows are features and columns are subjects
data = v.T

# IMPAC
if ds_flag=='impac':
  covars = {'site': np.array(data_train['participants_site']),
            'age': np.array(data_train['participants_age']),
            'sex': np.array(data_train['participants_sex']), 
            'diagnosis': np.array(labels_train)}
  covars = pd.DataFrame(covars)  

# ABIDE and SRPBS
if (ds_flag=='abide') or (ds_flag=='srpbs'):
  covars = {'site': np.array(data_train['SITE_ID']),
            'age': np.array(data_train['AGE_AT_SCAN']),
            'sex': np.array(data_train['SEX']),
            'diagnosis': np.array(labels_train)} 
  covars = pd.DataFrame(covars)   

# ADHD200
if ds_flag=='adhd200':
  covars = {'site': np.array(data_train['Site']),
            'age': np.array(data_train['Age']),
            'sex': np.array(data_train['Gender']),
            'diagnosis': np.array(labels_train)} 
  covars = pd.DataFrame(covars)  

# Categorical variables
categorical_cols = ['sex', 'diagnosis']
# Site variable
batch_col = 'site'

# Harmonization
data_combat = neuroCombat(dat=data,
    covars=covars,
    batch_col=batch_col,
    categorical_cols=categorical_cols)["data"]

# Convert to original format
v = data_combat.T
correlation_matrices_combat = v.reshape((v.shape[0], int(np.sqrt(v.shape[1])), int(np.sqrt(v.shape[1]))))

har_flag = 'combat'

[neuroCombat] Creating design matrix
[neuroCombat] Standardizing data across features
[neuroCombat] Fitting L/S model and finding priors
[neuroCombat] Finding parametric adjustments
[neuroCombat] Final adjustment of data


## CovBat

In [ ]:
# Convert matrices to vectors
s = correlation_matrices.shape
v = correlation_matrices.reshape((s[0],s[1]**2))

# Rows are features and columns are subjects
data = pd.DataFrame(v.T)

# IMPAC
if ds_flag=='impac':
  covars = {'site': np.array(data_train['participants_site']),
            'age': np.array(data_train['participants_age']),
            'sex': np.array(data_train['participants_sex']), 
            'diagnosis': np.array(labels_train)}
  covars = pd.DataFrame(covars)  
  mod = patsy.dmatrix("~ age + sex + diagnosis", covars, return_type="dataframe")

# ABIDE y SRPBS
if (ds_flag=='abide') or (ds_flag=='srpbs'):
  covars = {'site': np.array(data_train['SITE_ID']),
            'age': np.array(data_train['AGE_AT_SCAN']),
            'sex': np.array(data_train['SEX']),
            'diagnosis': np.array(labels_train)} 
  covars = pd.DataFrame(covars) 
  mod = patsy.dmatrix("~ age + sex + diagnosis", covars, return_type="dataframe")

# ADHD200
if ds_flag=='adhd200':
  covars = {'site': np.array(data_train['Site']),
            'age': np.array(data_train['Age']),
            'sex': np.array(data_train['Gender']),
            'diagnosis': np.array(labels_train)} 
  covars = pd.DataFrame(covars) 
  mod = patsy.dmatrix("~ age + sex + diagnosis", covars, return_type="dataframe")

# Harmonization
data_covbat = covbat.covbat(data, covars['site'], model=mod, numerical_covariates=['age'])

# Convert to original format
v = np.array(data_covbat.T)
correlation_matrices_covbat = v.reshape((v.shape[0], int(np.sqrt(v.shape[1])), int(np.sqrt(v.shape[1]))))

har_flag = 'covbat'

found 7 batches
found 1 numerical covariates...
	age
found 6 categorical variables:	diagnosis[T.1], diagnosis[T.2], diagnosis[T.3], diagnosis[T.333], diagnosis[T.666], sex
Standardizing Data across genes.
Fitting L/S model and finding priors
Finding parametric adjustments


Adjusting data


found 7 batches
found 0 numerical covariates...
found 0 categorical variables:	
Standardizing Data across genes.
Fitting L/S model and finding priors
Finding parametric adjustments


Adjusting data


## Traveling-subject method

In [ ]:
har_flag = 'traveling'

# Information Theory measures (Shannon-Fisher plane)

## Calculations (code in R)

In [ ]:
# Library that allows you to execute code in R
%load_ext rpy2.ipython

In [ ]:
%%R

# Shannon Entropy 'H'
shannon_entropy <- function(probs, normalized=FALSE)
{
  p = which(probs > 1e-30)
  entropy = -sum(probs[p]*log(probs[p]))
  if(normalized)
  {
    entropy = entropy/log(length(probs))
  }
  return(entropy)
}

# Fisher Information 'F'
fisher_information <- function(probs)
{
  N = length(probs)
  f = 1/2*sum((sqrt(probs[2:N]) - sqrt(probs[1:(N-1)]))^2)
  return(f)
}

In [ ]:
%%R

# Evaluate one network
EvaluateNetwork <- function(graph){

  # Simplify graph
  if(!is_simple(graph)){
    graph <- simplify(graph)  
  }
  
  # Number of nodes
  N <- vcount(graph)
  
  # Convert to adjacency matrix
  A <- as_adj(graph, attr = "weight")
  x <- as.matrix(A)
  
  # Sort the matrix optimally
  orow <- seriate(dist(x), method ="OLO")
  A <- seriation::permute(x, ser_permutation(orow, orow))  
  adjacencyMatrix <- Matrix(A)
  
  if(isSymmetric(adjacencyMatrix) == FALSE){
    stop("Graph is not undirected")
  }
  
  # Aditional metrics
  p <- edge_density(graph) # Density
  M <- ecount(graph) # Number of Edges 
  L <- mean_distance(graph) # Average path length
  CC <- transitivity(graph) # Clustering Coefficient
  K <- mean(degree(graph)) # Average Degree
  
  # Obtain H and F of each node
  results <- parRapply(cl, adjacencyMatrix,  function(m_row){ 
    row <- rep(x = 0, N)
    neighbors <- which(m_row > 0)  
    k <- sum(m_row)
    if(k > 0){
      row[neighbors] <- m_row[neighbors]/k
    }     

    # H and F
    entropy <- shannon_entropy(row)
    fisher <- fisher_information(row)   
    results <- cbind(entropy, fisher)

    return(results)
  })
  
  results <- matrix(results, nrow = 2, ncol = N)
  entropy <- results[1,]
  fisher <- results[2,]

  # H and F of the network
  H <- 1/(N*log(N - 1))*sum(entropy)
  H <- round(H, 3)
  FIM <- round(mean(fisher),3)
  
  result <- c(N, M, K, p, L, CC, -1, -1, -1, -1, -1, H, FIM)
  
  return(result)
}

## Libraries (code in R)

In [ ]:
%%R
install.packages("igraph")
install.packages("doParallel")
install.packages("seriation")

library(igraph)
library(Matrix)
library(parallel)
library(doParallel)
library(seriation)

##### Configuring parallelization #####
nCluster <- detectCores() - 2
cl <-makeCluster(2, type="FORK")
registerDoParallel(cl)

R[write to console]: Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

R[write to console]: trying URL 'https://cran.rstudio.com/src/contrib/igraph_1.3.2.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 2485997 bytes (2.4 MB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[writ

## Evaluation

In [ ]:
# Transformation of correlation matrices into adjacency matrices
@jit
def corr2adj(corr):

  # Number of subjects
  n_subjects = len(corr)
  # Create empty list
  adj = list()
  # Obtain the adjacency matrix for each subject
  for i in range(0, n_subjects): 
    intervals = corr[i].shape[0]
    adj_mat = np.zeros((intervals, intervals))
    for x in range(0, intervals):
      for y in range(0, intervals):

        if abs(corr[i][x,y]) > 0.5:
          adj_mat[x,y] = 1
        else:
          adj_mat[x,y] = 0        

    adj.append(adj_mat) 

  return adj 

# Calculate network metrics
def get_shannon_fisher(adj):

  %R -i adj  
  %R graph <- graph_from_adjacency_matrix(adj, mode = "undirected", weighted = TRUE)  
  %R results <- EvaluateNetwork(graph)
  %R -o results
  shannon_fisher = np.array([float(results[11]), float(results[12])])
  
  return shannon_fisher

In [ ]:
# Number of subjects
n_subjects = len(data_train)

# Obtain adjacency matrices
if (ds_flag=='srpbs') and (har_flag=='traveling'):
  adjacency_matrices = np.array(corr2adj(np.array(correlation_matrices_har)))
if har_flag=='none':
  adjacency_matrices = np.array(corr2adj(np.array(correlation_matrices)))
if har_flag=='combat':
  adjacency_matrices = np.array(corr2adj(np.array(correlation_matrices_combat)))
if har_flag=='covbat':
  adjacency_matrices = np.array(corr2adj(np.array(correlation_matrices_covbat)))

# Obtain Information Theory measures
shannon_fisher = list()
for i in range(0, n_subjects):
  sh = list()
  sh.append(get_shannon_fisher(adjacency_matrices[i]))
  shannon_fisher.append(sh)
  print(i)
shannon_fisher = np.array(shannon_fisher)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

## Save measures

In [ ]:
# Prepare the dataframe

s_f = shannon_fisher[:,0,:]
df = pd.DataFrame(s_f, columns=['shannon', 'fisher'])

site = list()
for i in range(0, len(subjects_data)):
  if (ds_flag=='impac') or (ds_flag=='srpbs'):
    s = subjects_data[i][0]        
  if ds_flag=='abide':
    s = subjects_data[i][5] 
  if ds_flag=='adhd200':
    s = subjects_data[i][1]   
  site.append(s)  

df["site"] = site

In [ ]:
# Save dataframe
df.to_csv(folder/'your_filename', index=False)

# Visualization

In [ ]:
# Read dataframe
df = pd.read_csv(folder/'your_filename')

In [ ]:
# Create a chart from a dataframe

selection = alt.selection_multi(fields=['site'])
color = alt.condition(selection,
                      alt.Color('site:N', legend=None, scale=alt.Scale(scheme='category20')),
                      alt.value('white'))

scatter = alt.Chart(df).mark_circle(size=80).encode(
    x=alt.X('shannon', title='Normalized Network Entropy', scale=alt.Scale(domain=(0,0.8))),
    y=alt.Y('fisher', title='Normalized Network Fisher Information', scale=alt.Scale(domain=(0.2,0.8))),
    color=color,
    opacity=alt.value(1),
    fillOpacity=alt.condition(selection, alt.value(1), alt.value(0)),
    stroke=alt.value('black'),
    strokeWidth=alt.value(0.7),
    strokeOpacity=alt.condition(selection, alt.value(1), alt.value(0.2))
).properties(
    width=650,
    height=650
).interactive()

legend = alt.Chart(df).mark_circle(size=80).encode(
    y=alt.Y('site:N', axis=alt.Axis(orient='right'), title='Acquisition site'),
    color=color,
    stroke=alt.value('black'),
    strokeWidth=alt.value(0.3)
).add_selection(
    selection
)

alt.hconcat(scatter, legend, center=True).configure_axis(
    labelFontSize=15,
    titleFontSize=25
)

alt.HConcatChart(...)

# Quantitative analysis

In [ ]:
def get_metrics(df, neg_log=False):
  sha = list()
  fis = list()
  for s in df.site.unique():
    val_s = df[df['site']==s]['shannon'].values
    val_f = df[df['site']==s]['fisher'].values
    sha.append(val_s)
    fis.append(val_f)
  return -np.log10([kruskal(*sha).pvalue, kruskal(*fis).pvalue])

def print_metrics(ds, har, p_s, p_f, atlas=None):
  print(20*'*', ds, 20*'*')
  if atlas != None:
    print('Atlas: ', atlas)
  print('Method: ', har)
  print('p-values: ',p_s,p_f)
  print('')
  return

In [ ]:
met = list()

# IMPAC

# ------------------------------------------------------------------------------

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/impac-msdl.csv')
m = get_metrics(df)
met.append(m)
print_metrics('IMPAC', 'No harmonization', *m, 'MSDL')

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/impac-msdl-combat.csv')
m = get_metrics(df)
met.append(m)
print_metrics('IMPAC', 'ComBat', *m, 'MSDL')

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/impac-msdl-gam.csv')
m = get_metrics(df)
met.append(m)
print_metrics('IMPAC', 'ComBat-GAM', *m, 'MSDL')

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/impac-msdl-covbat.csv')
m = get_metrics(df)
met.append(m)
print_metrics('IMPAC', 'CovBat', *m, 'MSDL')

# ------------------------------------------------------------------------------

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/impac-harvard.csv')
m = get_metrics(df)
met.append(m)
print_metrics('IMPAC', 'No harmonization', *m, 'Harvard-Oxford')

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/impac-harvard-combat.csv')
m = get_metrics(df)
met.append(m)
print_metrics('IMPAC', 'ComBat', *m, 'Harvard-Oxford')

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/impac-harvard-gam.csv')
m = get_metrics(df)
met.append(m)
print_metrics('IMPAC', 'ComBat-GAM', *m, 'Harvard-Oxford')

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/impac-harvard-covbat.csv')
m = get_metrics(df)
met.append(m)
print_metrics('IMPAC', 'CovBat', *m, 'Harvard-Oxford')

# ------------------------------------------------------------------------------

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/impac-basc064.csv')
m = get_metrics(df)
met.append(m)
print_metrics('IMPAC', 'No harmonization', *m, 'Basc064')

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/impac-basc064-combat.csv')
m = get_metrics(df)
met.append(m)
print_metrics('IMPAC', 'ComBat', *m, 'Basc064')

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/impac-basc064-gam.csv')
m = get_metrics(df)
met.append(m)
print_metrics('IMPAC', 'ComBat-GAM', *m, 'Basc064')

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/impac-basc064-covbat.csv')
m = get_metrics(df)
met.append(m)
print_metrics('IMPAC', 'CovBat', *m, 'Basc064')

# ------------------------------------------------------------------------------

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/impac-basc122.csv')
m = get_metrics(df)
met.append(m)
print_metrics('IMPAC', 'No harmonization', *m, 'Basc122')

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/impac-basc122-combat.csv')
m = get_metrics(df)
met.append(m)
print_metrics('IMPAC', 'ComBat', *m, 'Basc122')

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/impac-basc122-gam.csv')
m = get_metrics(df)
met.append(m)
print_metrics('IMPAC', 'ComBat-GAM', *m, 'Basc122')

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/impac-basc122-covbat.csv')
m = get_metrics(df)
met.append(m)
print_metrics('IMPAC', 'CovBat', *m, 'Basc122')

# ------------------------------------------------------------------------------

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/impac-basc197.csv')
m = get_metrics(df)
met.append(m)
print_metrics('IMPAC', 'No harmonization', *m, 'Basc197')

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/impac-basc197-combat.csv')
m = get_metrics(df)
met.append(m)
print_metrics('IMPAC', 'ComBat', *m, 'Basc197')

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/impac-basc197-gam.csv')
m = get_metrics(df)
met.append(m)
print_metrics('IMPAC', 'ComBat-GAM', *m, 'Basc197')

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/impac-basc197-covbat.csv')
m = get_metrics(df)
met.append(m)
print_metrics('IMPAC', 'CovBat', *m, 'Basc197')

# ------------------------------------------------------------------------------

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/impac-craddock.csv')
m = get_metrics(df)
met.append(m)
print_metrics('IMPAC', 'No harmonization', *m, 'Craddock')

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/impac-craddock-combat.csv')
m = get_metrics(df)
met.append(m)
print_metrics('IMPAC', 'ComBat', *m, 'Craddock')

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/impac-craddock-gam.csv')
m = get_metrics(df)
met.append(m)
print_metrics('IMPAC', 'ComBat-GAM', *m, 'Craddock')

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/impac-craddock-covbat.csv')
m = get_metrics(df)
met.append(m)
print_metrics('IMPAC', 'CovBat', *m, 'Craddock')

# ------------------------------------------------------------------------------

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/impac-power.csv')
m = get_metrics(df)
met.append(m)
print_metrics('IMPAC', 'No harmonization', *m, 'Power')

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/impac-power-combat.csv')
m = get_metrics(df)
met.append(m)
print_metrics('IMPAC', 'ComBat', *m, 'Power')

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/impac-power-gam.csv')
m = get_metrics(df)
met.append(m)
print_metrics('IMPAC', 'ComBat-GAM', *m, 'Power')

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/impac-power-covbat.csv')
m = get_metrics(df)
met.append(m)
print_metrics('IMPAC', 'CovBat', *m, 'Power')

******************** IMPAC ********************
Atlas:  MSDL
Method:  No harmonization
p-values:  131.94946357991662 24.793642270354358

******************** IMPAC ********************
Atlas:  MSDL
Method:  ComBat
p-values:  0.24184238573648592 4.586435328924532

******************** IMPAC ********************
Atlas:  MSDL
Method:  ComBat-GAM
p-values:  0.20570891143327727 5.665449082307004

******************** IMPAC ********************
Atlas:  MSDL
Method:  CovBat
p-values:  0.07548948949291433 1.0117218655984839

******************** IMPAC ********************
Atlas:  Harvard-Oxford
Method:  No harmonization
p-values:  137.05986373437352 34.341923754931365

******************** IMPAC ********************
Atlas:  Harvard-Oxford
Method:  ComBat
p-values:  0.6628679709717066 6.263681267034916

******************** IMPAC ********************
Atlas:  Harvard-Oxford
Method:  ComBat-GAM
p-values:  1.2373879066654474 9.079721701370726

******************** IMPAC ********************
Atlas:

In [ ]:
met = list()

# ABIDE

# ------------------------------------------------------------------------------

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/abide-aal.csv')
m = get_metrics(df)
met.append(m)
print_metrics('ABIDE', 'No harmonization', *m, 'AAL')

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/abide-aal-combat.csv')
m = get_metrics(df)
met.append(m)
print_metrics('ABIDE', 'ComBat', *m, 'AAL')

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/abide-aal-gam.csv')
m = get_metrics(df)
met.append(m)
print_metrics('ABIDE', 'ComBat-GAM', *m, 'AAL')

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/abide-aal-covbat.csv')
m = get_metrics(df)
met.append(m)
print_metrics('ABIDE', 'CovBat', *m, 'AAL')

# ------------------------------------------------------------------------------

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/abide-cc200.csv')
m = get_metrics(df)
met.append(m)
print_metrics('ABIDE', 'No harmonization', *m, 'Craddock200')

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/abide-cc200-combat.csv')
m = get_metrics(df)
met.append(m)
print_metrics('ABIDE', 'ComBat', *m, 'Craddock200')

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/abide-cc200-gam.csv')
m = get_metrics(df)
met.append(m)
print_metrics('ABIDE', 'ComBat-GAM', *m, 'Craddock200')

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/abide-cc200-covbat.csv')
m = get_metrics(df)
met.append(m)
print_metrics('ABIDE', 'CovBat', *m, 'Craddock200')

# ------------------------------------------------------------------------------

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/abide-cc400.csv')
m = get_metrics(df)
met.append(m)
print_metrics('ABIDE', 'No harmonization', *m, 'Craddock400')

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/abide-cc400-combat.csv')
m = get_metrics(df)
met.append(m)
print_metrics('ABIDE', 'ComBat', *m, 'Craddock400')

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/abide-cc400-gam.csv')
m = get_metrics(df)
met.append(m)
print_metrics('ABIDE', 'ComBat-GAM', *m, 'Craddock400')

# df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/abide-cc400-covbat.csv')
# m = get_metrics(df)
# met.append(m)
# print_metrics('ABIDE', 'CovBat', *m, 'Craddock400')

# ------------------------------------------------------------------------------

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/abide-dosenbach160.csv')
m = get_metrics(df)
met.append(m)
print_metrics('ABIDE', 'No harmonization', *m, 'Dosenbach')

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/abide-dosenbach160-combat.csv')
m = get_metrics(df)
met.append(m)
print_metrics('ABIDE', 'ComBat', *m, 'Dosenbach')

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/abide-dosenbach160-gam.csv')
m = get_metrics(df)
met.append(m)
print_metrics('ABIDE', 'ComBat-GAM', *m, 'Dosenbach')

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/abide-dosenbach160-covbat.csv')
m = get_metrics(df)
met.append(m)
print_metrics('ABIDE', 'CovBat', *m, 'Dosenbach')

# ------------------------------------------------------------------------------

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/abide-ez.csv')
m = get_metrics(df)
met.append(m)
print_metrics('ABIDE', 'No harmonization', *m, 'Eickhoff-Zilles')

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/abide-ez-combat.csv')
m = get_metrics(df)
met.append(m)
print_metrics('ABIDE', 'ComBat', *m, 'Eickhoff-Zilles')

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/abide-ez-gam.csv')
m = get_metrics(df)
met.append(m)
print_metrics('ABIDE', 'ComBat-GAM', *m, 'Eickhoff-Zilles')

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/abide-ez-covbat.csv')
m = get_metrics(df)
met.append(m)
print_metrics('ABIDE', 'CovBat', *m, 'Eickhoff-Zilles')

# ------------------------------------------------------------------------------

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/abide-ho.csv')
m = get_metrics(df)
met.append(m)
print_metrics('ABIDE', 'No harmonization', *m, 'Harvard-Oxford')

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/abide-ho-combat.csv')
m = get_metrics(df)
met.append(m)
print_metrics('ABIDE', 'ComBat', *m, 'Harvard-Oxford')

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/abide-ho-gam.csv')
m = get_metrics(df)
met.append(m)
print_metrics('ABIDE', 'ComBat-GAM', *m, 'Harvard-Oxford')

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/abide-ho-covbat.csv')
m = get_metrics(df)
met.append(m)
print_metrics('ABIDE', 'CovBat', *m, 'Harvard-Oxford')

# ------------------------------------------------------------------------------

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/abide-tt.csv')
m = get_metrics(df)
met.append(m)
print_metrics('ABIDE', 'No harmonization', *m, 'Talaraich-Tournoux')

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/abide-tt-combat.csv')
m = get_metrics(df)
met.append(m)
print_metrics('ABIDE', 'ComBat', *m, 'Talaraich-Tournoux')

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/abide-tt-gam.csv')
m = get_metrics(df)
met.append(m)
print_metrics('ABIDE', 'ComBat-GAM', *m, 'Talaraich-Tournoux')

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/abide-tt-covbat.csv')
m = get_metrics(df)
met.append(m)
print_metrics('ABIDE', 'CovBat', *m, 'Talaraich-Tournoux')

******************** ABIDE ********************
Atlas:  AAL
Method:  No harmonization
p-values:  29.131300300223952 19.49027491420303

******************** ABIDE ********************
Atlas:  AAL
Method:  ComBat
p-values:  0.008697427627224034 2.3545537628813618

******************** ABIDE ********************
Atlas:  AAL
Method:  ComBat-GAM
p-values:  0.050113327845153266 2.309587344793073

******************** ABIDE ********************
Atlas:  AAL
Method:  CovBat
p-values:  1.5785356893554588e-05 0.9264216025303278

******************** ABIDE ********************
Atlas:  Craddock200
Method:  No harmonization
p-values:  41.584695205019266 34.4433707655307

******************** ABIDE ********************
Atlas:  Craddock200
Method:  ComBat
p-values:  0.2575955224042766 7.527392061581657

******************** ABIDE ********************
Atlas:  Craddock200
Method:  ComBat-GAM
p-values:  0.4528645407978996 7.6683856418278085

******************** ABIDE ********************
Atlas:  Craddoc

In [ ]:
# SRPBS
df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/srpbs.csv')
m = get_metrics(df)
print_metrics('SRPBS', 'No harmonization', *m)

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/srpbs-traveling.csv')
m = get_metrics(df)
print_metrics('SRPBS', 'Traveling dataset', *m)

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/srpbs-combat.csv')
m = get_metrics(df)
print_metrics('SRPBS', 'ComBat', *m)

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/srpbs-gam.csv')
m = get_metrics(df)
print_metrics('SRPBS', 'ComBat-GAM', *m)

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/srpbs-covbat.csv')
m = get_metrics(df)
print_metrics('SRPBS', 'CovBat', *m)

******************** SRPBS ********************
Method:  No harmonization
p-values:  77.116069565349 27.69965579943464

******************** SRPBS ********************
Method:  Traveling dataset
p-values:  72.1493576437605 18.049448441218377

******************** SRPBS ********************
Method:  ComBat
p-values:  0.2853962051700903 4.3488121768735715

******************** SRPBS ********************
Method:  ComBat-GAM
p-values:  0.5973140124714698 5.187921287229803

******************** SRPBS ********************
Method:  CovBat
p-values:  2.1374169319392493 4.813382099959848



In [ ]:
met = list()

# ADHD-200

# ------------------------------------------------------------------------------

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/adhd200-aal.csv')
m = get_metrics(df)
met.append(m)
print_metrics('ADHD-200', 'No harmonization', *m, 'AAL')

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/adhd200-aal-combat.csv')
m = get_metrics(df)
met.append(m)
print_metrics('ADHD-200', 'ComBat', *m, 'AAL')

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/adhd200-aal-gam.csv')
m = get_metrics(df)
met.append(m)
print_metrics('ADHD-200', 'ComBat-GAM', *m, 'AAL')

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/adhd200-aal-covbat.csv')
m = get_metrics(df)
met.append(m)
print_metrics('ADHD-200', 'CovBat', *m, 'AAL')

# ------------------------------------------------------------------------------

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/adhd200-cc200.csv')
m = get_metrics(df)
met.append(m)
print_metrics('ADHD-200', 'No harmonization', *m, 'Craddock200')

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/adhd200-cc200-combat.csv')
m = get_metrics(df)
met.append(m)
print_metrics('ADHD-200', 'ComBat', *m, 'Craddock200')

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/adhd200-cc200-gam.csv')
m = get_metrics(df)
met.append(m)
print_metrics('ADHD-200', 'ComBat-GAM', *m, 'Craddock200')

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/adhd200-cc200-covbat.csv')
m = get_metrics(df)
met.append(m)
print_metrics('ADHD-200', 'CovBat', *m, 'Craddock200')

# ------------------------------------------------------------------------------

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/adhd200-cc400.csv')
m = get_metrics(df)
met.append(m)
print_metrics('ADHD-200', 'No harmonization', *m, 'Craddock400')

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/adhd200-cc400-combat.csv')
m = get_metrics(df)
met.append(m)
print_metrics('ADHD-200', 'ComBat', *m, 'Craddock400')

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/adhd200-cc400-gam.csv')
m = get_metrics(df)
met.append(m)
print_metrics('ADHD-200', 'ComBat-GAM', *m, 'Craddock400')

# df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/adhd200-cc400-covbat.csv')
# m = get_metrics(df)
# met.append(m)
# print_metrics('ADHD-200', 'CovBat', *m, 'Craddock400')

# ------------------------------------------------------------------------------

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/adhd200-ez.csv')
m = get_metrics(df)
met.append(m)
print_metrics('ADHD-200', 'No harmonization', *m, 'Eickhoff-Zilles')

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/adhd200-ez-combat.csv')
m = get_metrics(df)
met.append(m)
print_metrics('ADHD-200', 'ComBat', *m, 'Eickhoff-Zilles')

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/adhd200-ez-gam.csv')
m = get_metrics(df)
met.append(m)
print_metrics('ADHD-200', 'ComBat-GAM', *m, 'Eickhoff-Zilles')

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/adhd200-ez-covbat.csv')
m = get_metrics(df)
met.append(m)
print_metrics('ADHD-200', 'CovBat', *m, 'Eickhoff-Zilles')

# ------------------------------------------------------------------------------

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/adhd200-ho.csv')
m = get_metrics(df)
met.append(m)
print_metrics('ADHD-200', 'No harmonization', *m, 'Harvard-Oxford')

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/adhd200-ho-combat.csv')
m = get_metrics(df)
met.append(m)
print_metrics('ADHD-200', 'ComBat', *m, 'Harvard-Oxford')

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/adhd200-ho-gam.csv')
m = get_metrics(df)
met.append(m)
print_metrics('ADHD-200', 'ComBat-GAM', *m, 'Harvard-Oxford')

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/adhd200-ho-covbat.csv')
m = get_metrics(df)
met.append(m)
print_metrics('ADHD-200', 'CovBat', *m, 'Harvard-Oxford')

# ------------------------------------------------------------------------------

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/adhd200-tt.csv')
m = get_metrics(df)
met.append(m)
print_metrics('ADHD-200', 'No harmonization', *m, 'Talaraich-Tournoux')

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/adhd200-tt-combat.csv')
m = get_metrics(df)
met.append(m)
print_metrics('ADHD-200', 'ComBat', *m, 'Talaraich-Tournoux')

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/adhd200-tt-gam.csv')
m = get_metrics(df)
met.append(m)
print_metrics('ADHD-200', 'ComBat-GAM', *m, 'Talaraich-Tournoux')

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/adhd200-tt-covbat.csv')
m = get_metrics(df)
met.append(m)
print_metrics('ADHD-200', 'CovBat', *m, 'Talaraich-Tournoux')

******************** ADHD-200 ********************
Atlas:  AAL
Method:  No harmonization
p-values:  102.82306041096572 58.03424214165506

******************** ADHD-200 ********************
Atlas:  AAL
Method:  ComBat
p-values:  8.783137820113488 17.995336235436874

******************** ADHD-200 ********************
Atlas:  AAL
Method:  ComBat-GAM
p-values:  8.717733784509322 21.144425749106922

******************** ADHD-200 ********************
Atlas:  AAL
Method:  CovBat
p-values:  5.476372273424307 15.118533242650093

******************** ADHD-200 ********************
Atlas:  Craddock200
Method:  No harmonization
p-values:  114.14079566218412 75.40428058339317

******************** ADHD-200 ********************
Atlas:  Craddock200
Method:  ComBat
p-values:  12.893809755429118 34.50211461048108

******************** ADHD-200 ********************
Atlas:  Craddock200
Method:  ComBat-GAM
p-values:  13.46627562531602 36.76619580148486

******************** ADHD-200 ********************
At